<style>.container { width:100% !important; }</style>

# Kubernetes

Dados la problemática que presenta la escabilidad en entornos de múltiples servidores, se creó Kubernetes para permitir una administración de estos, permitiendo automatizar el despliegue, ajustar la escala y ayudar con el manejo de contenedores

### Un ejemplo

Si durante la noche un servidor sufre un desperfecto, kubernetes se encargará de montar los contenedores perdidos en otros servidores para mantener el sistema funcionando

Kubernetes tiene tres partes muy importantes:
- Controladores: Se preocupan de decidir donde estarán los contenedores, los requerimientos que necesita cada uno.
- Workers: Máquinas que ejecutan pods, los nodos trabajan en red, es decir, si un nodo cae no importa, el controlador puede levantar los pods perdidos en otra máquina y la red continúa trabajando.
- etcd: Aquí se almacena la información del estado de elemento miembro del clúster.

<img src="images/arquitectura" width="800" height="800" align="center"/>


#### Kubelet
Agente que se ejecuta en cada nodo de un clúster. Se asegura de que los contenedores estén corriendo en un pod.  
El agente kubelet toma un conjunto de especificaciones de Pod, llamados PodSpecs, que han sido creados por Kubernetes y garantiza que los contenedores descritos en ellos estén funcionando y en buen estado.

#### Scheduler
Componente del plano de control que está pendiente de los Pods que no tienen ningún nodo asignado y seleciona uno donde ejecutarlo.  
Para decidir en qué nodo se ejecutará el pod, se tienen en cuenta diversos factores: requisitos de recursos, restricciones de hardware/software/políticas, afinidad y anti-afinidad, localización de datos dependientes, entre otros.

#### Controller-manager

Componente del plano de control que ejecuta los controladores de Kubernetes.

Lógicamente cada controlador es un proceso independiente, pero para reducir la complejidad, todos se compilan en un único binario y se ejecuta en un mismo proceso.

<img src="images/arquitectura" width="800" height="800" align="center"/>

<font size="3">Estos controladores incluyen:</font>
  
   **<font size="3">Controlador de nodos</font>**<font size="3">: es el responsable de detectar y responder cuándo un nodo deja de funcionar.</font>  
   **<font size="3">Controlador de replicación</font>**<font size="3">: es el responsable de mantener el número correcto de pods para cada controlador de replicación del sistema.</font>  
   **<font size="3">Controlador de endpoints</font>**<font size="3">: construye el objeto Endpoints, es decir, hace una unión entre los Services y los Pods.</font>  
   **<font size="3">Controladores de tokens y cuentas de servicio</font>**<font size="3">: crean cuentas y tokens de acceso a la API por defecto para los nuevos Namespaces.</font>

# Montando un clúster básico con Minikube



Chequear que la virtualización está habilitada en la bios del pc:  
```egrep --color 'vmx|svm' /proc/cpuinfo```

Instalar un operador de maquinas virtuales (VirtualBox, KVM, etc)  

Instalar kubectl:
```sudo pacman -S kubectl```

Instalar minikube:
```sudo pacman -S minikube```


In [57]:
minikube start

😄  minikube v1.13.1 en Arch 20.1.1
✨  Automatically selected the kvm2 driver
👍  Starting control plane node minikube in cluster minikube
🔥  Creating kvm2 VM (CPUs=2, Memory=2400MB, Disk=20000MB) ...
🐳  Preparando Kubernetes v1.19.2 en Docker 19.03.12...
🔎  Verifying Kubernetes components...
🌟  Enabled addons: default-storageclass, storage-provisioner
🏄  Done! kubectl is now configured to use "minikube" by default


In [58]:
kubectl get nodes

NAME       STATUS   ROLES    AGE     VERSION
minikube   Ready    master   2m32s   v1.19.2


In [60]:
minikube status

minikube
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured



## Conceptos



### Namespace  
Básicamente crean clústers virtuales dentro del mismo clúster físico, ocupan la misma infraestructura pero cada uno permanece aislado del otro, por ejemplo esto es ideal para tener clústers de desarrollo y producción.  

In [61]:
kubectl get namespaces

NAME              STATUS   AGE
default           Active   2m17s
kube-node-lease   Active   2m56s
kube-public       Active   2m57s
kube-system       Active   3m


### Pods  

Es la unidad mas pequeña que maneja kubernetes, consiste en un conjunto de contenedores donde funciona alguna aplicación.  

#### Consideraciones
No use Pods desnudos (es decir, Pods que no estén vinculados a ReplicaSet o Deployment) si puede evitarlo. Los pods desnudos no se reprogramarán en caso de falla de un nodo.  

Una deployment crea un ReplicaSet para garantizar que la cantidad deseada de pods siempre esté disponible, y especifica una estrategia para reemplazar los pods (como RollingUpdate), es casi siempre preferible a crear pods directamente. Un job también puede ser apropiado.

In [62]:
kubectl get pods

No resources found in default namespace.


### Servicios  

Es una abstracción que define un conjunto de pods que implementan un micro-servicio.  

Existen 3 tipos de servicios:



ClusterIP:  
Solo permite el acceso interno entre distintos servicios. Es el tipo por defecto. Podemos acceder desde el exterior con la instrucción kubectl proxy, puede de ser gran ayuda para los desarrolladores.  

![title](images/ClusterIP)

NodePort:  
Abre un puerto, para que el servicio sea accesible desde el exterior. Por defecto el puerto generado está en el rango de 30000:40000. Para acceder usamos la ip del servidor master del cluster y el puerto asignado.

![title](images/NodePort)

LoadBalancer:  
Este tipo sólo esta soportado en servicios de cloud público (GKE, AKS o AWS). El proveedor asignara un recurso de balanceo ed carga para el acceso a los servicios. si usamos un cloud privado, como OpenSatck necesitaremos un plugin para configurar el funcionamiento.

![title](images/LoadBalancer)

#### Consideraciones
Define y usa etiquetas que identifiquen atributos semanticos de tu aplicación o deployment, por ejemplo:
```{ app: myapp, tier: frontend, phase: test, deployment: v3 }```. Puedes usar estas etiquetas para seleccionar apropiadamente los pods para otros recursos; por ejemplo, si quiero que un servicio seleccione todos los pods ```tier: frontend``` o todos los componentes ```phase: test``` de la ```app: myapp```.

In [63]:
kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   2m26s


### Manifiestos  

Los recursos pueden crearse directamente desde la línea de comandos, pero esto no es recomendable en producción debido a los potenciales errores humanos al realizar estas tareas, por esta razón es mejor utilizar los manifiestos.

Los manifiestos son documentos utilizados para crear, modificar o eliminar recursos de Kubernetes, como pods, despliegues, servicios e ingresos.

In [14]:
cat yaml/00-namespace.yaml

kind: Namespace
apiVersion: v1
metadata:
  name: testing 


In [65]:
kubectl apply -f yaml/00-namespace.yaml

namespace/testing created


In [66]:
kubectl get namespaces

NAME              STATUS   AGE
default           Active   2m54s
kube-node-lease   Active   3m33s
kube-public       Active   3m34s
kube-system       Active   3m37s
testing           Active   4s


In [17]:
cat yaml/01-wordpress-service.yaml

kind: Service           #pod, deployment, service, namespace, replicaset, StatefulSet, DaemonSet, 
                        #Job o lo que sea que vayas a describir
apiVersion: v1
metadata:
  name: wordpress       #nombre del servicio en este caso
spec:
  type: NodePort        #Aquí se indica si se trabajará con las ip internas o se abrirá un puerto 
                        #para desplegarlo externamente, según vimos, NodePort ordena abrir un puerto
                        #en el worker para poder acceder externamente a este servicio
  ports:
  - port:80             #Se indica el puerto que se abre en el worker
    targetPort: 80      #Es el puerto al que apunta el servicio en cualquier pod que tenga el rol
                        #wordpress
    nodePort: 30000     #Es el puerto del nodo que apunta al 80 del pod
  selector:
    role: wordpress

# Si se desea, se puede especificar otro recurso en el manifiesto separandolos por guiones
#---


In [67]:
kubectl -n testing apply -f yaml/01-wordpress-service.yaml
kubectl -n testing get services

service/wordpress created
NAME        TYPE       CLUSTER-IP    EXTERNAL-IP   PORT(S)        AGE
wordpress   NodePort   10.109.82.5   <none>        80:30000/TCP   1s


In [31]:
cat yaml/02-wordpress-rc.yaml

apiVersion: v1
kind: ReplicationController
metadata:
  name: wordpress
spec:
  replicas: 1
  template:
    metadata:
      labels:
        role: wordpress
    spec:
      containers:
      - name: wordpress
        image: wordpress:php7.2-apache
        imagePullPolicy: IfNotPresent
        ports:
        - containerPort: 80


La imagen se descarga por defecto desde DockerHub; si quieres usar un registry alternativo, debes indicar la URL completa al recurso.

In [84]:
kubectl -n testing apply -f yaml/02-wordpress-rc.yaml
kubectl -n testing get pods

replicationcontroller/wordpress unchanged
NAME              READY   STATUS    RESTARTS   AGE
wordpress-7tb7f   1/1     Running   0          24m


In [85]:
kubectl -n testing delete pod wordpress-7tb7f
kubectl -n testing get pods

pod "wordpress-7tb7f" deleted
NAME              READY   STATUS              RESTARTS   AGE
wordpress-hrk4n   0/1     ContainerCreating   0          16s


Ahora podemos ver que nuestro servicio funciona correctamente

In [78]:
kubectl get nodes -o wide

NAME       STATUS   ROLES    AGE    VERSION   INTERNAL-IP      EXTERNAL-IP   OS-IMAGE               KERNEL-VERSION   CONTAINER-RUNTIME
minikube   Ready    master   7m6s   v1.19.2   192.168.39.225   <none>        Buildroot 2019.02.11   4.19.114         docker://19.3.12


In [81]:
kubectl -n testing get services

NAME        TYPE       CLUSTER-IP    EXTERNAL-IP   PORT(S)        AGE
wordpress   NodePort   10.109.82.5   <none>        80:30000/TCP   11m


In [82]:
minikube service -n testing wordpress --url

http://192.168.39.225:30000


In [80]:
minikube dashboard

🔌  Enabling dashboard ...
🤔  Verifying dashboard health ...
🚀  Launching proxy ...
🤔  Verifying proxy health ...
🎉  Opening http://127.0.0.1:46429/api/v1/namespaces/kubernetes-dashboard/services/http:kubernetes-dashboard:/proxy/ in your default browser...

